# 佐賀県の人口推移

In [ ]:
import pandas
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
#url = 'http://data.bodik.jp/dataset/77e0cc66-c15d-4473-b3df-2664fe8e2e63/resource/8dc71515-526a-4168-866c-05d2cc8dad7b/download/jinkou.xlsx'
url = 'jinkou.xlsx'

In [ ]:
#エクセル中の和暦ラベルと西暦の対応付け
wareki ={'大正9年':1920, '昭和5年':1930, '10年':1935, '15年':1940,
        '20年':1945, '25年':1950, '30年':1955, '35年':1960,'40年':1965,
        '45年':1970, '50年':1975, '55年':1980, '60年':1985, '平成2年':1990,
        '7年':1995, '12年':2000, '17年':2005, '22年':2010, '27年':2015}

In [ ]:
def getIndex(url:str)->dict[str,str]:
    """
    URLで指定されたExcelで、V列に対応する適切な行名を取得する
    """
    dd = pandas.read_excel(url, header = 3,  usecols = 'A:B,V',
        index_col = 2, skiprows = [4, 18, 27], skipfooter = 8) 
    newIndex:dict[str,str] = dict()
    for k in dd.index:
        v = dd['項'][k]
        if type(v) != str:
            v = dd['目'][k]
        newIndex[k] = v.strip()
    return newIndex

In [ ]:
def readData(url:str)->pandas.DataFrame:
    """
    urlで指定されたExcelファイルを読む
    """
    data = pandas.read_excel(url, header = 3, usecols = 'C:V',
             index_col = 19, skiprows = [4, 18, 27], skipfooter = 8) 
    keys = getIndex(url)
    return data.rename(columns = wareki, index = keys)

In [ ]:
def plotSub(yearList:list[int], df:pandas.DataFrame,ax:plt.Axes):
    """
    yearListで指定された年の棒グラフを描画
    """
    #必要なデータに成形する：対象となる年のデータを取得し、1/1000にする
    population :list[pandas.Series]= [
        df.loc['0～14歳'].filter(yearList)/1000,
        df.loc['15～64歳'].filter(yearList)/1000,
        df.loc['65歳以上'].filter(yearList)/1000
    ]
    #積み上げの基点となるSeries
    bottom = pandas.Series([0]*len(yearList), index = yearList)
    color = ['g', 'y', 'r']
    label = ['年少', '生産年齢', '老年']
    for k in range(len(population)):
        population[k].plot.bar(ax = ax, bottom = bottom, 
            tick_label = yearList, color = color[k], label = label[k])
        bottom += population[k]


作図前処理

In [ ]:
def prePlot()->plt.Axes:
    fig, ax =plt.subplots(figsize = (15, 15), facecolor = 'white')
    plt.rcParams["font.size"] = 32
    ax.set_title('佐賀県の人口 (国勢調査から)') #図タイトル
    ax.set_xlabel('年')
    ax.set_ylabel('人口 (千人)')
    ax.set_ylim(0, 1200)
    return ax

メイン

In [ ]:
#Excelの内容をDataFrameへ
data = readData(url)
ax = prePlot()

yearList = list()
year = 1945
while year < 2020:
    yearList.append(year)
    year += 10
plotSub(yearList, data, ax)
plt.legend(loc = 'best') #凡例

plt.savefig('SagaPopulationStackedBar.pdf') #画像保存
plt.show() #画像表示
